# Ligand Preparation for PyRosetta

#### Presenter: Jason Klima (klimaj@uw.edu)

### For this tutorial, we need scripts from the Rosetta codebase that are not packaged with  `conda`-deployed `pyrosetta`, so we need to have cloned the RosettaCommons github repository.
#### Recommended method:
>mkdir ~/git && cd ~/git

>curl -OL https://raw.github.com/RosettaCommons/rosetta_clone_tools/master/get_rosetta.sh && bash get_rosetta.sh

### For this tutorial, we will be docking ATP into cAMP-dependent protein kinase catalytic subunit alpha isoform 1 from *Mus musculus* using three different methods to exemplify the diversity of strategies available in PyRosetta:
### 1. High-resolution ligand docking using `DockMCMProtocol` with the `ligand.wts` scorefunction and Gasteiger ligand partial charges
### 2. Global ligand docking using `XMLObjects` with the `ref2015.wts` scorefunction and AM1-BCC ligand partial charges
### 3. Global ligand docking using `pyrosetta.distributed` with the `beta_genpot.wts` scorefunction and AM1-BCC ligand partial charges

In [1]:
import logging
logging.basicConfig(level=logging.INFO)
import os
import py3Dmol
import sys

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from IPython.display import IFrame

### Components of Rosetta residue full-atom and centroid `.params` files

#### Rosetta `.params` files store a variety of chemical and geometric information used to define the shape and chemical connectivity of a ligand or amino acid building block or other small molecule.

In [2]:
IFrame("https://graylab.jhu.edu/pyrosetta/downloads/documentation/PyRosetta_Workshops_Appendix_B.pdf", width=1200, height=800)

### Some potentially useful notes:

- Some ligand `.params` files already exist in `~/anaconda3/envs/pyrosetta-code-school/lib/python3.6/site-packages/pyrosetta/database/chemical/residue_type_sets`
 - If you want to use them, uncomment lines for ligand `.params` files in `~/anaconda3/envs/pyrosetta-code-school/lib/python3.6/site-packages/pyrosetta/database/chemical/residue_type_sets/fa_standard/residue_types.txt`, then use `pyrosetta.init()` without specifying the `-extra_res_fa` flag because PyRosetta will find the `.params` file in the database.
 - Warning: this method is *not* recommended, because next time you run `conda update pyrosetta -c rosettacommons` your changes to the database will be lost. A possible workaround would be to permanantly modify `RosettaCommons/main/database/chemical/residue_type_sets/fa_standard/residue_types.txt` on github after seeking peer review. If you must use these files, the best solution would be to manually load them into PyRosetta using `pyrosetta.init("-extra_res_fa {}".format(ligand_params_path))`.
 - Warning: as we will see, these extant `.params` files are *most likely* only compatible with `talaris2014.wts` or previous scorefunctions.
- Upon re-weighting the newer energy functions (`ref2015.wts`, `beta_nov16.wts`, and `beta_genpot.wts`) energy function weights were optimized on computational models of ligands with partial charges generated with the AM1-BCC force field in Amber’s `antechamber` software suite.


### Quick notes about where to obtain ligand conformers:

- If the ligand exists in the PDB in a medium-to-high resolution structure, it is probably a good conformer to use for generating partial charges.

- Sometimes you only want to model a ligand in a specific conformer (i.e. planar state or extended conformation) even though it is not the conformer with the lowest internal energy. In this case, you want to generate ligand partial charges for this specific conformer.

- If the ligand of interest does not exist in the PDB, several software packages allow you to make `.pdb` files. Examples include:

 - CSD Mercury --> Edit Molecule…

 - Avogadro

 - PubChem Sketcher V2.4 (https://pubchem.ncbi.nlm.nih.gov/edit2/index.html) -> copy the SMILES string to process with the `rdkit` module in python -> output `.pdb` file
 
### Ultimately, the conformer you choose will affect the partial charges generated, and the partial charges generated will influence the Rosetta `total_score`, `interfE` score, etc., affecting packing and minimization, so choose wisely!

### Let's retreive the `.pdb` file containing ATP bound to cAMP-dependent protein kinase catalytic subunit alpha isoform 1 from *Mus musculus*:

In [3]:
input_dir = os.path.join(os.getcwd(), "input")
if not os.path.isdir(input_dir):
    os.mkdir(input_dir)
os.chdir(input_dir)
!cp ~/git/Rosetta/main/source/src/python/PyRosetta/src/demo/data/test_lig.pdb .

### Now extract the ATP molecule to a separate `.pdb` file:

In [4]:
!grep "^HETATM" test_lig.pdb > ATP.pdb

### To generate partial charges, we need to know the formal charge of the ligand.
### Always manually inspect the ligand for which you are generating partial charges to ascertain formal charge - do not rely on PubChem or otherwise as protonation states can differ!

In [5]:
mol = open(os.path.join(os.getcwd(), "ATP.pdb")).read()
viewer = py3Dmol.view(600, 300)
viewer.addModel(mol, "pdb")
viewer.setStyle({"stick": {}})
viewer.zoomTo()
viewer.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

### Since this ligand came from a crystal structure, there aren't any hydrogens on the ATP molecule. We need to add them either in `babel` using `-h -p <pH>` flags or manually in a molecular editing program before generating partial charges.
### Also, in order to generate `.params` files for Rosetta, we need to convert the `.pdb`/`.mdl`/`.mol`/`.cif` etc. file of the ligand into a `.mol2` file. 
### We can combine these steps making use of the `babel` executable:

In [6]:
babel =  os.path.join(os.path.dirname(sys.executable), "babel")
!{babel} -ipdb ATP.pdb -omol2 ATP.mol2 -h -p 7.0

1 molecule converted
45 audit log messages 


### Now let's make sure the hydrogens look the way we want them to look.

In [7]:
mol = open(os.path.join(os.getcwd(), "ATP.mol2")).read()
viewer = py3Dmol.view(600, 300)
viewer.addModel(mol, "mol2")
viewer.setStyle({"stick": {}})
viewer.zoomTo()
viewer.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

### We can see that the formal charge is -4.

### From `ATP.mol2`, we will generate the following:
### 1. An `ATP.gasteiger.fa.params` file with Gasteiger partial charges with the standard atom type set (namely "fa_standard") using the `molfile_to_params.py` script to be used with the `talaris2014.wts` scorefunction and earlier.
### 2. An `ATP.am1-bcc.fa.params` file with AM1-BCC partial charges with the standard atom type set (namely "fa_standard") using the `molfile_to_params.py` script to be used with the `ref2015.wts`,  `ref2015_cart.wts`, `beta_nov16.wts` and `beta_nov16_cart.wts` scorefunctions.
### 3. An `ATP.am1-bcc.gp.params` file with AM1-BCC partial charges with the generic atom type set (namely "gen_pot") using the `mol2genparams.py` script to be used with `beta_genpot.wts` scorefunction and beyond. Note: still being actively developed!

### For more information on the difference between atom type sets, see: https://www.rosettacommons.org/docs/latest/GenericAtomtypes

## Generating `ATP.gasteiger.fa.params`:

### For this approach we will use the `~Rosetta/main/source/scripts/python/public/molfile_to_params.py` script to generate full-atom and centroid `.params` files for modeling ligands in Rosetta and PyRosetta.
### The input file is a `.mol2` file of the ligand, not `.pdb`/`.mdl`/`.mol`/`.cif` etc. files
### Check the available command line options:

In [8]:
molfile_to_params = "~/git/Rosetta/main/source/scripts/python/public/molfile_to_params.py"
!python2.7 {molfile_to_params} -h

Usage: molfile_to_params.py [flags] { INPUT.mol | INPUT.sdf | INPUT.mol2 }

Converts a small molecule in an MDL Molfile with "M SPLT" and "M ROOT"
records into a series of .params residue definition files for Rosetta.
Also writes out the ligand conformation as PDB HETATMs.

If an SD file is given as input instead, the first entry is used for
generating topology / parameter files, and they all are used for
generating PDB-style coordinates in separate, numbered files.
These multiple files can optionally be concatenated into a single file,
which can then be specified with an additional PDB_ROTAMERS line in the
.params file to include the extra conformations as ligand rotamers.
Multiple models may also be supplied in MOL2 format, which does not support
M ROOT and M SPLT records but does allow for partial charges.
File type is deduced from the extension.

To divide a ligand into fragments by "breaking" bonds (optional):
M SPLT atom_no1 atom_no2

To specify a neighbor atom for a ligand fragm

### Running `molfile_to_params.py`:

In [9]:
!python2.7 {molfile_to_params} -n ATP -p ATP -c --conformers-in-one-file --keep-names --extra_torsion_output --clobber --chain=X --recharge=-4 ATP.mol2

Centering ligands at (  10.506,    9.569,    1.503)
Atom names contain duplications -- renaming all atoms.
Ignoring partial charges in file; net charge set to -4.000
Total naive charge -5.805, desired charge -4.000, offsetting all atoms by 0.042
Average 43.0 atoms (31.0 non-H atoms) per fragment
(Proteins average 15.5 atoms (7.8 non-H atoms) per residue)
Wrote params file ATP.fa.params
Wrote params file ATP.cen.params


### Check the files that were created!

In [10]:
!ls -ltrh

total 414K
-rw-r--r-- 1 klimaj baker 5.8K May  2 15:55 ATP.am1-bcc.fa.params
-rw-r--r-- 1 klimaj baker 3.4K May  2 15:55 sqm.pdb
-rw-r--r-- 1 klimaj baker 3.5K May  2 15:55 ATP.am1-bcc.fa.pdb
-rw-r--r-- 1 klimaj baker 2.8K May  2 15:55 ATP.am1-bcc_0001.pdb
-rw-r--r-- 1 klimaj baker 9.8K May  2 15:55 ATOMTYPE.INF
-rw-r--r-- 1 klimaj baker 4.9K May  2 15:55 ANTECHAMBER_BOND_TYPE.AC
-rw-r--r-- 1 klimaj baker 4.9K May  2 15:55 ANTECHAMBER_AC.AC0
-rw-r--r-- 1 klimaj baker 4.9K May  2 15:55 ANTECHAMBER_AC.AC
-rw-r--r-- 1 klimaj baker 2.4K May  2 15:55 sqm.in
-rw-r--r-- 1 klimaj baker  14K May  2 15:55 sqm.out
-rw-r--r-- 1 klimaj baker 5.6K May  2 15:55 ATP.am1-bcc.params
-rw-r--r-- 1 klimaj baker 5.6K May  2 15:55 ATP.am1-bcc.gp.params
-rw-r--r-- 1 klimaj baker 4.9K May  2 15:55 ANTECHAMBER_BOND_TYPE.AC0
-rw-r--r-- 1 klimaj baker 4.8K May  2 15:55 ATP.am1-bcc.cen.params
-rw-r--r-- 1 klimaj baker 5.8K May  2 15:55 ATP.gasteiger.fa.params
-rw-r--r-- 1 klimaj baker 2.9K May  2 15:55 ATP.am1-bcc

### Our "fa_standard" full atom `.params` file is `ATP.fa.params`, and our "fa_standard" centroid `.params` file is `ATP.cen.params`.
### Inspect the file `ATP.fa.params`:

In [11]:
!cat ATP.fa.params

NAME ATP
IO_STRING ATP Z
TYPE LIGAND
AA UNK
ATOM  C1  CH2   X   -0.14
ATOM  O10 OH    X   -0.62
ATOM  P3  Phos  X   1.54
ATOM  O7  OOC   X   -0.72
ATOM  O9  OH    X   -0.62
ATOM  P2  Phos  X   1.54
ATOM  O4  OOC   X   -0.72
ATOM  O5  OOC   X   -0.72
ATOM  O6  OH    X   -0.62
ATOM  P1  Phos  X   1.54
ATOM  O3  OOC   X   -0.72
ATOM  O1  OOC   X   -0.72
ATOM  O2  OOC   X   -0.72
ATOM  O8  OOC   X   -0.72
ATOM  C2  CH1   X   -0.05
ATOM  O11 OH    X   -0.62
ATOM  C5  CH1   X   -0.05
ATOM  N1  Npro  X   -0.33
ATOM  C6  aroC  X   -0.07
ATOM  N2  Nhis  X   -0.49
ATOM  C7  aroC  X   -0.07
ATOM  C8  aroC  X   -0.07
ATOM  N3  NH2O  X   -0.43
ATOM  H10 Hpol  X   0.47
ATOM  H11 Hpol  X   0.47
ATOM  N4  Nhis  X   -0.49
ATOM  C9  aroC  X   -0.07
ATOM  N5  Nhis  X   -0.49
ATOM  C10 aroC  X   -0.07
ATOM  H12 Haro  X   0.16
ATOM  H9  Haro  X   0.16
ATOM  C4  CH1   X   -0.05
ATOM  C3  CH1   X   -0.05
ATOM  O12 OH    X   -0.62
ATOM  H5  Hpol  X   0.47
ATOM  H4  Hapo  X   0.14
ATOM  O13 OH    X   -0.62
ATO

### *Gasteiger partial charges were generated on-the-fly!*

### Pay attention to lines beginning with `CHI` and `PROTON_CHI`. These $\chi$ torsions indiciate which dihedral angles can rotate during minimization. If you want to turn any of these off for specific applications, either comment out the line(s) or delete the line(s).

### If your `.mol2` file had many conformer models in it, the `--conformers-in-one-file` flag would have produced a file called `ATP.conf.pdb`, which the packer could use to repack the ligand (similar to the Dunbrack rotamer library for side chains). You would then specify the full path at the bottom of the `ATP.gasteiger.fa.params` file as in:
>PDB_ROTAMERS /full/path/to/ATP.conf.pdb

### However, since we only have one conformer for ATP, the ligand can only minimize to change conformations.

### Because we passed the flag `--extra_torsion_output` to `molfile_to_params.py`, we outputted the files `ATP.fa.tors` and `ATP.cen.tors`.
### Inpect `ATP.fa.tors`:

In [11]:
!cat ATP.fa.tors

ATP  C7   N2   C6   N1   0  80  2
ATP  C7   N2   C6   H9   0  80  2
ATP  N1   N2   C6   H9   0  80  2
ATP  C6   N2   C7   C8   0  80  2
ATP  C6   N2   C7   C10  0  80  2
ATP  C8   N2   C7   C10  0  80  2
ATP  N2   C6   N1   C10  0  80  2
ATP  N2   C6   N1   C5   0  80  2
ATP  N2   C6   N1   H9   0  80  2
ATP  C10  C6   N1   C5   0  80  2
ATP  C10  C6   N1   H9   0  80  2
ATP  C5   C6   N1   H9   0  80  2
ATP  N3   C8   C7   N2   0  80  2
ATP  N3   C8   C7   N4   0  80  2
ATP  N3   C8   C7   C10  0  80  2
ATP  N2   C8   C7   N4   0  80  2
ATP  N2   C8   C7   C10  0  80  2
ATP  N4   C8   C7   C10  0  80  2
ATP  N3   C8   N4   C7   0  80  2
ATP  N3   C8   N4   C9   0  80  2
ATP  C7   C8   N4   C9   0  80  2
ATP  N2   C7   C10  C8   0  80  2
ATP  N2   C7   C10  N1   0  80  2
ATP  N2   C7   C10  N5   0  80  2
ATP  C8   C7   C10  N1   0  80  2
ATP  C8   C7   C10  N5   0  80  2
ATP  N1   C7   C10  N5   0  80  2
ATP  C6   N1   C10  C7   0  80  2
ATP  C6   N1   C10  C5   0  80  2
ATP  C6   N1  

### If you pass the Rosetta command line flag `-score:extra_improper_file ATP.fa.tors` to `pyrosetta.init()`, then PyRosetta will constrain the listed torsions in this file during minimization. The columns in this file correspond to:
### [3 letter residue name] [Atom 1] [Atom 2] [Atom 3] [Atom 4] [Dihedral Angle] [Constraint Weight] [Periodicity]

### Because we passed the flag `--keep-names` to `molfile_to_params.py`, the ATP atom names in `test_lig.pdb`  still match those in `ATP.fa.params`. Even though `test_lig.pdb` does not contain the added hydrogens, Rosetta will automatically build the remaining hydrogens upon loading the ligand into the pose because `ATP.fa.params` contains the bond connectivity information.

### Rename `ATP.fa.params` for future use:

In [12]:
!cp ATP.fa.params ATP.gasteiger.fa.params

## Generating `ATP.am1-bcc.fa.params`:

### Now we will generate AM1-BCC partial charges for ATP using `antechamber`. This quantum mechanical calculation takes a bit longer but only needs to be done once per ligand.

> ### "The goal of the charge model is to produce atomic charges that emulate the HF/6-31G* electrostatic potential (ESP) of a molecule. Underlying electronic structure features, including formal charge and electron delocalization, are first captured by AM1 population charges; simple additive bond charge corrections (BCCs) are then applied to these AM1 atomic charges to produce the AM1-BCC charges."
Reference: Jakalian, A. , Jack, D. B. and Bayly, C. I. (2002), Fast, efficient generation of high‐quality atomic charges. AM1‐BCC model: II. Parameterization and validation. *J. Comput. Chem.*, 23: 1623-1641. 

In [ ]:
antechamber =  os.path.join(os.path.dirname(sys.executable), "antechamber")
%time !{antechamber} -i ATP.mol2 -fi mol2 -o ATP.am1-bcc.mol2 -fo mol2 -c bcc -at sybyl -nc -4


Welcome to antechamber 17.3: molecular input file processor.

acdoctor mode is on: check and diagnosis problems in the input file.
-- Check Format for mol2 File --
   Status: pass
-- Check Unusual Elements --
   Status: pass
-- Check Open Valences --
         the connectivity (2) for atom type (N.ar) defined in CORR_NAME_TYPE.DAT.
But, you may safely ignore the warnings if your molecule
         uses atom names or element names as atom types.
-- Check Geometry --
      for those bonded   
      for those not bonded   
   Status: pass
-- Check Weird Bonds --
   Status: pass
-- Check Number of Units --
   Status: pass
acdoctor mode has completed checking the input file.

Info: Total number of electrons: 260; net charge: -4

Running: /home/klimaj/anaconda3/envs/pyrosetta-code-school/bin/sqm -O -i sqm.in -o sqm.out


### Make sure the output file `ATP.am1-bcc.mol2` was successfully generated!

In [14]:
!ls -ltrh

total 233K
-rw-r--r-- 1 klimaj baker 222K Feb 21 11:03 test_lig.pdb
-rw-r--r-- 1 klimaj baker 2.5K Feb 21 11:03 ATP.pdb
-rw-r--r-- 1 klimaj baker 4.4K Feb 21 11:03 ATP.mol2
-rw-r--r-- 1 klimaj baker 3.5K Feb 21 11:03 ATP.fa.pdb
-rw-r--r-- 1 klimaj baker 5.8K Feb 21 11:03 ATP.fa.params
-rw-r--r-- 1 klimaj baker 1.4K Feb 21 11:03 ATP.fa.tors
-rw-r--r-- 1 klimaj baker 2.9K Feb 21 11:03 ATP.cen.pdb
-rw-r--r-- 1 klimaj baker 4.9K Feb 21 11:03 ATP.cen.params
-rw-r--r-- 1 klimaj baker 1.1K Feb 21 11:03 ATP.cen.tors
-rw-r--r-- 1 klimaj baker 5.8K Feb 21 11:03 ATP.gasteiger.fa.params
-rw-r--r-- 1 klimaj baker 4.9K Feb 21 11:03 ANTECHAMBER_BOND_TYPE.AC0
-rw-r--r-- 1 klimaj baker 4.9K Feb 21 11:03 ANTECHAMBER_BOND_TYPE.AC
-rw-r--r-- 1 klimaj baker 4.9K Feb 21 11:03 ANTECHAMBER_AC.AC0
-rw-r--r-- 1 klimaj baker 2.4K Feb 21 11:03 sqm.in
-rw-r--r-- 1 klimaj baker 4.9K Feb 21 11:03 ANTECHAMBER_AC.AC
-rw-r--r-- 1 klimaj baker  14K Feb 21 11:12 sqm.out
-rw-r--r-- 1 klimaj baker 3.4K Feb 21 11:12 sqm.pdb

### It is a good idea at this step to make sure the output file `ATP.am1-bcc.mol2` looks as it should.

In [15]:
mol = open(os.path.join(os.getcwd(), "ATP.am1-bcc.mol2")).read()
viewer = py3Dmol.view(600, 300)
viewer.addModel(mol, "mol2")
viewer.setStyle({"stick": {}})
viewer.zoomTo()
viewer.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

### We will use the `~Rosetta/main/source/scripts/python/public/molfile_to_params.py` script to generate full-atom and centroid `.params` files, this time *without* the `--recharge` flag!

In [16]:
!python2.7 {molfile_to_params} -n ATP -p ATP.am1-bcc -c --conformers-in-one-file --keep-names --extra_torsion_output --clobber --chain=X ATP.am1-bcc.mol2

Centering ligands at (  10.506,    9.569,    1.503)
Partial charges already fully assigned, no changes made; net charge -4.001
Average 43.0 atoms (31.0 non-H atoms) per fragment
(Proteins average 15.5 atoms (7.8 non-H atoms) per residue)
Wrote params file ATP.am1-bcc.fa.params
Wrote params file ATP.am1-bcc.cen.params


In [17]:
!ls -ltrh

total 236K
-rw-r--r-- 1 klimaj baker 222K Feb 21 11:03 test_lig.pdb
-rw-r--r-- 1 klimaj baker 2.5K Feb 21 11:03 ATP.pdb
-rw-r--r-- 1 klimaj baker 4.4K Feb 21 11:03 ATP.mol2
-rw-r--r-- 1 klimaj baker 3.5K Feb 21 11:03 ATP.fa.pdb
-rw-r--r-- 1 klimaj baker 5.8K Feb 21 11:03 ATP.fa.params
-rw-r--r-- 1 klimaj baker 1.4K Feb 21 11:03 ATP.fa.tors
-rw-r--r-- 1 klimaj baker 2.9K Feb 21 11:03 ATP.cen.pdb
-rw-r--r-- 1 klimaj baker 4.9K Feb 21 11:03 ATP.cen.params
-rw-r--r-- 1 klimaj baker 1.1K Feb 21 11:03 ATP.cen.tors
-rw-r--r-- 1 klimaj baker 5.8K Feb 21 11:03 ATP.gasteiger.fa.params
-rw-r--r-- 1 klimaj baker 4.9K Feb 21 11:03 ANTECHAMBER_BOND_TYPE.AC0
-rw-r--r-- 1 klimaj baker 4.9K Feb 21 11:03 ANTECHAMBER_BOND_TYPE.AC
-rw-r--r-- 1 klimaj baker 4.9K Feb 21 11:03 ANTECHAMBER_AC.AC0
-rw-r--r-- 1 klimaj baker 2.4K Feb 21 11:03 sqm.in
-rw-r--r-- 1 klimaj baker 4.9K Feb 21 11:03 ANTECHAMBER_AC.AC
-rw-r--r-- 1 klimaj baker  14K Feb 21 11:12 sqm.out
-rw-r--r-- 1 klimaj baker 3.4K Feb 21 11:12 sqm.pdb

### Again, ATP atom names were preserved during the AM1-BCC calculation and `.params` file generation (due to the `--keep-names` flag), so `ATP.am1-bcc.fa.params` is still compatible with `test_lig.pdb` (i.e. again, even though the ATP molecule in `test_lig.pdb` is missing the hydrogens, PyRosetta will re-build them from the `.params` file).

## Generating `ATP.am1-bcc.gp.params`:

### New atom typing for using the latest & greatest scorefunction! 

### Generate a `.params` file using the new `mol2genparams.py` script. Warning: actively being developed!
> python mol2genparams.py [-s mol2file or -l mol2filelist]

#### Note: this is applicable only for using the `beta_genpot.wts` and `beta_genpot_cart.wts` (currently identical to `beta.wts` and `beta_cart.wts`, but may change) scorefunctions and beyond (i.e. and *most likely* any scorefunciton developed *after* `beta_nov16.wts`).

In [18]:
mol2genparams = "~/git/Rosetta/main/source/scripts/python/public/generic_potential/mol2genparams.py"
%run {mol2genparams} -s ATP.am1-bcc.mol2

ATP.am1-bcc.mol2


In [19]:
%ls -ltrh

total 237K
-rw-r--r-- 1 klimaj baker 222K Feb 21 11:03 test_lig.pdb
-rw-r--r-- 1 klimaj baker 2.5K Feb 21 11:03 ATP.pdb
-rw-r--r-- 1 klimaj baker 4.4K Feb 21 11:03 ATP.mol2
-rw-r--r-- 1 klimaj baker 3.5K Feb 21 11:03 ATP.fa.pdb
-rw-r--r-- 1 klimaj baker 5.8K Feb 21 11:03 ATP.fa.params
-rw-r--r-- 1 klimaj baker 1.4K Feb 21 11:03 ATP.fa.tors
-rw-r--r-- 1 klimaj baker 2.9K Feb 21 11:03 ATP.cen.pdb
-rw-r--r-- 1 klimaj baker 4.9K Feb 21 11:03 ATP.cen.params
-rw-r--r-- 1 klimaj baker 1.1K Feb 21 11:03 ATP.cen.tors
-rw-r--r-- 1 klimaj baker 5.8K Feb 21 11:03 ATP.gasteiger.fa.params
-rw-r--r-- 1 klimaj baker 4.9K Feb 21 11:03 ANTECHAMBER_BOND_TYPE.AC0
-rw-r--r-- 1 klimaj baker 4.9K Feb 21 11:03 ANTECHAMBER_BOND_TYPE.AC
-rw-r--r-- 1 klimaj baker 4.9K Feb 21 11:03 ANTECHAMBER_AC.AC0
-rw-r--r-- 1 klimaj baker 2.4K Feb 21 11:03 sqm.in
-rw-r--r-- 1 klimaj baker 4.9K Feb 21 11:03 ANTECHAMBER_AC.AC
-rw-r--r-- 1 klimaj baker  14K Feb 21 11:12 sqm.out
-rw-r--r-- 1 klimaj baker 3.4K Feb 21 11:12 sqm.pdb

### Two output files were generated: `ATP.am1-bcc_0001.pdb` and `ATP.am1-bcc.params`. Notice that the atom typing in `ATP.am1-bcc.params` is different than before!

In [20]:
!cat ATP.am1-bcc.params

NAME LG1
IO_STRING LG1 Z
TYPE LIGAND
AA UNK
ATOM P1   PG3  X     1.448
ATOM O1   OG2  X    -0.947
ATOM O2   OG2  X    -0.947
ATOM O3   OG2  X    -0.947
ATOM P2   PG3  X     1.678
ATOM O4   OG2  X    -0.918
ATOM O5   OG2  X    -0.918
ATOM O6   OG3  X    -0.798
ATOM P3   PG3  X     1.608
ATOM O7   OG2  X    -0.670
ATOM O8   OG2  X    -0.670
ATOM O9   OG3  X    -0.789
ATOM O10  OG3  X    -0.606
ATOM C1   CSp  X     0.168
ATOM C2   CS1  X     0.096
ATOM O11  Oet  X    -0.434
ATOM C3   CSp  X     0.112
ATOM O12  Ohx  X    -0.674
ATOM C4   CSp  X     0.114
ATOM O13  Ohx  X    -0.620
ATOM C5   CS1  X     0.241
ATOM N1   NGb  X    -0.274
ATOM C6   CD1  X     0.349
ATOM N2   Nim  X    -0.775
ATOM C7   CR   X    -0.111
ATOM C8   CR   X     0.529
ATOM N3   Nam2 X    -0.889
ATOM N4   Nim  X    -0.778
ATOM C9   CR   X     0.557
ATOM N5   Nim  X    -0.707
ATOM C10  CR   X     0.387
ATOM H    HC   X     0.071
ATOM H1   HC   X     0.071
ATOM H2   HC   X     0.062
ATOM H3   HC   X     0.079
ATOM H4   H

### Also notice that the 3 letter ligand name `ATP` was changed to `LG1` by default. Therefore, in order for the `.params` file to be compatible with the ATP molecule in `test_lig.pdb` in PyRosetta, we need to rename the 3-letter molecule name manually (Note: this will change in the future).
### Also rename `ATP.am1-bcc.params` to  `ATP.am1-bcc.gp.params` for future use:

In [21]:
!cp ATP.am1-bcc.params ATP.am1-bcc.gp.params
!sed -i s/LG1/ATP/g ATP.am1-bcc.gp.params

### Further reading: https://graylab.jhu.edu/pyrosetta/downloads/scripts/demo/D120_Ligand_interface.py